In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import wandb
from xv import io
import torch
from tqdm.notebook import tqdm
from datetime import datetime
import os

In [ ]:
seg_run_id = "qoijsx0h"
dmg_run_id = "d6wvpmgq"

In [ ]:
from datetime import datetime
import os
test_dir = '../../datasets/xview/test'
submission_dir = f"../submissions/{seg_run_id}_{dmg_run_id}_{str(datetime.now()).replace(' ', '_').replace(':', '-')}"
os.mkdir(submission_dir)
print(submission_dir)

In [ ]:
from xv.io import Config

In [ ]:
seg_run_path = f"xvr-hlt/sky-eye-full/{seg_run_id}"
conf_file = wandb.restore('config.yaml', run_path=seg_run_path, replace=True).name
state_file = wandb.restore('state_dict.pth', run_path=seg_run_path, replace=True).name

conf = Config(conf_file)
model, preprocess_fn = io.load_segmentation_model(conf, state_file)

In [ ]:
import ttach as tta
model = tta.SegmentationTTAWrapper(model, tta.aliases.d4_transform(), merge_mode='mean')
model = model.eval().cuda()

In [ ]:
from glob import glob
pre_files = glob(f"{test_dir}/images/*pre*")

In [ ]:
from PIL import Image
import numpy as np

with torch.no_grad():
    for f in tqdm(pre_files):
        i = io.load_img(f, preprocess_fn)
        out = model(i.cuda())[0][0] > 0.
        out = out.cpu().numpy().astype(np.uint8)
        fid = f.split('_')[-1].replace(".png", "")
        Image.fromarray(out).save(f"{submission_dir}/test_localization_{fid}_prediction.png")

In [ ]:
dmg_run_path = f"xvr-hlt/building-damage/{dmg_run_id}"
dmg_conf_file = wandb.restore('config.yaml', run_path=dmg_run_path, replace=True).name
dmg_state_file = wandb.restore('state_dict.pth', run_path=dmg_run_path, replace=True).name

In [ ]:
from glob import glob 
from PIL import Image
import numpy as np
post_files = glob(f"{test_dir}/images/*post*")
dmg_conf = Config(dmg_conf_file)

In [ ]:
from xv.tta import BoxClassifierTTA
dmg_model = io.load_damage_model(dmg_conf, dmg_state_file)
dmg_model = BoxClassifierTTA(dmg_model)
dmg_model = dmg_model.eval().cuda()

In [ ]:
import cv2
from imantics import Polygons

for f in tqdm(post_files):
    fid = f.split('_')[-1].replace(".png", "")
    mask_file = f"{submission_dir}/test_localization_{fid}_prediction.png"
    msk = np.array(Image.open(mask_file))
    polys = Polygons.from_mask(msk)
    
    polypoints = polys.points
    if polypoints:
        with torch.no_grad():
            boxes = torch.Tensor([[min(p[:,0]), min(p[:,1]), max(p[:,0]), max(p[:,1])] for p in polypoints])
            inp = io.load_dmg_img(f)
            out = dmg_model(inp.cuda(), [boxes.cuda()])
            classes = (out.argmax(1) + 1).cpu().numpy()
    else:
        classes = []

    msk_dmg = np.zeros((1024, 1024), dtype=np.uint8)
    
    for poly, cls in zip(polypoints, classes):
        cv2.fillPoly(msk_dmg, [poly], int(cls))

    Image.fromarray(msk_dmg).save(f"{submission_dir}/test_damage_{fid}_prediction.png")

In [ ]:
_split = submission_dir.rfind('/') + 1
root, subdir = submission_dir[:_split], submission_dir[_split:]

! cd "{root}" && zip -r "{subdir}.zip" "{subdir}/"

In [ ]:
!rm config.yaml state_dict.pth